In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import metrics
pd.set_option('display.max_columns', None)

In [2]:
Test_df = pd.read_csv('Test.csv', index_col=[0])
Train_df = pd.read_csv('Train_Acitivity_Tracker_Labeled.csv', index_col=[0])

In [3]:
Train_df.head()

,0,1,2,filename,user,experiment,index,labels
0,1.020833,-0.125000,0.105556,acc_exp01_user01.txt,1,1,250,5
1,1.025000,-0.125000,0.101389,acc_exp01_user01.txt,1,1,251,5
2,1.020833,-0.125000,0.104167,acc_exp01_user01.txt,1,1,252,5
3,1.016667,-0.125000,0.108333,acc_exp01_user01.txt,1,1,253,5
4,1.018056,-0.127778,0.108333,acc_exp01_user01.txt,1,1,254,5


In [4]:
Test_df

,0,1,2,user,experiment
Index,,,,,
0,1.006944,-0.152778,-0.095833,57,28
1,1.011111,-0.130556,-0.105556,57,28
2,1.011111,-0.130556,-0.105556,57,28
3,1.005556,-0.144444,-0.087500,57,28
4,0.993056,-0.161111,-0.084722,57,28
...,...,...,...,...,...
70393,0.880556,-0.390278,-0.156944,61,30
70394,0.834722,-0.358333,-0.098611,61,30
70395,0.802778,-0.329167,-0.104167,61,30


In [5]:
Train_df['Shift_x'] = Train_df['0'].shift()
Train_df['Shift_y'] = Train_df['1'].shift()
Train_df['Shift_z'] = Train_df['2'].shift()
Train_df.head()

,0,1,2,filename,user,experiment,index,labels,Shift_x,Shift_y,Shift_z
0,1.020833,-0.125000,0.105556,acc_exp01_user01.txt,1,1,250,5,NaN,NaN,NaN
1,1.025000,-0.125000,0.101389,acc_exp01_user01.txt,1,1,251,5,1.020833,-0.125,0.105556
2,1.020833,-0.125000,0.104167,acc_exp01_user01.txt,1,1,252,5,1.025000,-0.125,0.101389
3,1.016667,-0.125000,0.108333,acc_exp01_user01.txt,1,1,253,5,1.020833,-0.125,0.104167
4,1.018056,-0.127778,0.108333,acc_exp01_user01.txt,1,1,254,5,1.016667,-0.125,0.108333


In [12]:
# Difference function om het verschil tussen de vorige waarde en de huidige waarde te berekenen.
Train_df['Diff_x'] = Train_df['0'] - Train_df['Shift_x']
Train_df['Diff_y'] = Train_df['1'] - Train_df['Shift_y']
Train_df['Diff_z'] = Train_df['2'] - Train_df['Shift_z']

In [18]:
Train_df.groupby(['user', 'experiment', '1hz_ind']).agg({'0':'mean','1':'mean','2':'mean','labels':'median'})


0         1         2  labels
user experiment 1hz_ind                                      
1    1          249.0    1.020833 -0.125000  0.105556     5.0
                252.0    1.020833 -0.125000  0.104630     5.0
                255.0    1.018519 -0.127315  0.104630     5.0
                258.0    1.018982 -0.122222  0.096759     5.0
                261.0    1.020370 -0.120370  0.098148     5.0
...                           ...       ...       ...     ...
56   28         19632.0  1.144445 -0.272685 -0.214352     2.0
                19635.0  1.344444 -0.296296 -0.378704     2.0
                19638.0  0.788426 -0.355556 -0.162037     2.0
                19641.0  0.811574 -0.359259 -0.228704     2.0
                19644.0  0.968056 -0.258333 -0.420833     2.0

[248762 rows x 4 columns]

In [13]:
Train_df['1hz_ind'] = Train_df['index']
Train_df['1hz_ind'] = 3 * round(Train_df['index'] / 3)
Train_df.head()

,0,1,2,filename,user,experiment,index,labels,Shift_x,Shift_y,Shift_z,Diff_x,1hz_ind,Diff_y,Diff_z
0,1.020833,-0.125000,0.105556,acc_exp01_user01.txt,1,1,250,5,NaN,NaN,NaN,NaN,249.0,NaN,NaN
1,1.025000,-0.125000,0.101389,acc_exp01_user01.txt,1,1,251,5,1.020833,-0.125,0.105556,0.004167,252.0,0.000000,-0.004167
2,1.020833,-0.125000,0.104167,acc_exp01_user01.txt,1,1,252,5,1.025000,-0.125,0.101389,-0.004167,252.0,0.000000,0.002778
3,1.016667,-0.125000,0.108333,acc_exp01_user01.txt,1,1,253,5,1.020833,-0.125,0.104167,-0.004167,252.0,0.000000,0.004167
4,1.018056,-0.127778,0.108333,acc_exp01_user01.txt,1,1,254,5,1.016667,-0.125,0.108333,0.001389,255.0,-0.002778,0.000000


In [19]:
Train_df.groupby(['user','experiment', '1hz_ind','Diff_x','Diff_y','Diff_z','Shift_x','Shift_y','Shift_z' ]).head()

,0,1,2,filename,user,experiment,index,labels,Shift_x,Shift_y,Shift_z,Diff_x,1hz_ind,Diff_y,Diff_z
0,1.020833,-0.125000,0.105556,acc_exp01_user01.txt,1,1,250,5,NaN,NaN,NaN,NaN,249.0,NaN,NaN
1,1.025000,-0.125000,0.101389,acc_exp01_user01.txt,1,1,251,5,1.020833,-0.125000,0.105556,0.004167,252.0,0.000000,-0.004167
2,1.020833,-0.125000,0.104167,acc_exp01_user01.txt,1,1,252,5,1.025000,-0.125000,0.101389,-0.004167,252.0,0.000000,0.002778
3,1.016667,-0.125000,0.108333,acc_exp01_user01.txt,1,1,253,5,1.020833,-0.125000,0.104167,-0.004167,252.0,0.000000,0.004167
4,1.018056,-0.127778,0.108333,acc_exp01_user01.txt,1,1,254,5,1.016667,-0.125000,0.108333,0.001389,255.0,-0.002778,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745211,0.636111,-0.433333,-0.065278,acc_exp56_user28.txt,56,28,19639,2,0.762500,-0.348611,-0.144444,-0.126389,19638.0,-0.084722,0.079167
745212,0.690278,-0.431944,-0.108333,acc_exp56_user28.txt,56,28,19640,2,0.636111,-0.433333,-0.065278,0.054167,19641.0,0.001389,-0.043056
745213,0.829167,-0.370833,-0.223611,acc_exp56_user28.txt,56,28,19641,2,0.690278,-0.431944,-0.108333,0.138889,19641.0,0.061111,-0.115278
745214,0.915278,-0.275000,-0.354167,acc_exp56_user28.txt,56,28,19642,2,0.829167,-0.370833,-0.223611,0.086111,19641.0,0.095833,-0.130556


GROUPBY AANPASSEN MET NIEUWE KOLOMMEN BIJ DIFF EN SHIFT

In [ ]:
GroupTrain_df.to_csv('GroupTrain_df.csv')

In [ ]:
Train_df = pd.read_csv('GroupTrain_df.csv')
Train_df

In [ ]:
X=Train_df[['0', '1', '2']]  # Features
y=Train_df['labels']  # Labels
X_test = Test_df[['0', '1', '2']]

# Split dataset into training set and test set
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

neigh = KNeighborsClassifier(n_neighbors=1000, weights='distance', algorithm='auto').fit(X, y)
y_predicted = neigh.predict(X_test)
Test_df["labels"] = y_predicted.astype(int) #X veranderen Test_df voor uploaden
Test_df['labels'].to_csv('Voorspelling.csv')
